In [1]:
#Importing essential libraries
import pandas as pd
import numpy as np
import csv
import json
from itertools import islice
from collections import OrderedDict
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import nltk
from glob import glob
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from tqdm import tqdm
import pickle
import math
from sklearn.model_selection import train_test_split
import operator
from tqdm import tqdm
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('corpus')
! pip install git+https://github.com/LIAAD/yake
import yake

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Error loading corpus: Package 'corpus' not found in index
  Cloning https://github.com/LIAAD/yake to /tmp/pip-req-build-kzkw8aj4
  Running command git clone -q https://github.com/LIAAD/yake /tmp/pip-req-build-kzkw8aj4


In [2]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
file1 = open("/content/drive/MyDrive/Tweelink_Dataset/twitter_base_preprocessed.pkl", "rb")
df = pickle.load(file1)
file1.close()

In [5]:
df.head()

,author id,created_at,geo,country,country_code,place_full_name,place_name,place_type,id,lang,...,source,tweet,hashtags,sensitive,urls,context_text,context_probability,context_type,Preprocessed_Data,Date_Only
0,1952510090.0,2022-02-14 00:11:44+00:00,,,,,,,1493015060943454208.0,en,...,Twitter for Android,ð¥ð¥ð¥ð¥ð¥ð¥ð¥â¤µï¸â¤µï¸â¤µï¸...,"CommonSenseGunLaws,GunControlNow,GunSafes,GunS...",False,https://t.co/LjkEup24Dk,,0.0,,[ð¥ð¥ð¥ð¥ð¥ð¥ð¥â¤µï¸â¤µï¸â¤µï¸...,2022-02-14
1,20453105.0,2022-02-14 00:04:56+00:00,,,,,,,1493013352938934272.0,en,...,Twitter for Android,"It's too bad these guys are afraid of needles,...","ClownConvoy,FreeDumbConvoy,OttawaOccupied",False,https://t.co/MmFzuFjIDR,,0.0,,"[bad, guy, afraid, needle, twinrix, would, pre...",2022-02-14
2,1492660754998247424.0,2022-02-14 00:39:55+00:00,,,,,,,1493022155239534336.0,en,...,Twitter for Android,Cowboy dressed as #Furries now available at th...,"Furries,RamRanch,OttawaOccupied",False,https://t.co/GBuBCUtpXe,,0.0,,"[cowboy, dressed, furries, available, ramranch...",2022-02-14
3,117812637.0,2022-02-14 00:20:05+00:00,,,,,,,1493017163485044736.0,en,...,Twitter for iPhone,We blocked these trucks from entering the dntn...,"Riverside,OttawaOccupied",False,https://t.co/KzLkBZvjgD,,0.0,,"[blocked, truck, entering, dntn, core, riversi...",2022-02-14
4,1490044524604928000.0,2022-02-14 00:10:26+00:00,,,,,,,1493014734903382016.0,en,...,Twitter for iPhone,Krista is very pleased with how the @RCMPONT r...,"FluTruxKlanGoHome,OttawaOccupied,kkkonvoy,Otta...",False,https://t.co/kAm5KNugdA,,0.0,,"[krista, pleased, rcmpont, responded, maskless...",2022-02-14


In [6]:
u_base_hashtag = input("Enter base hashtag: ")
u_time = input("Enter time: ")
u_location = input("Enter Location: ")

Enter base hashtag: hijab
Enter time: 2022-02-19
Enter Location: india


In [7]:
import datetime
tweet_query = []
format = '%Y-%m-%d'
u_present_date = datetime.datetime.strptime(u_time, format)
u_prev_date = u_present_date - datetime.timedelta(days=1)
u_next_date = u_present_date + datetime.timedelta(days=1)
df_query = df.loc[df['hashtags'].str.contains(u_base_hashtag) & df['Date_Only'].isin([str(u_present_date.date()), str(u_prev_date.date()), str(u_next_date.date())])]

In [8]:
df_query.head()

,author id,created_at,geo,country,country_code,place_full_name,place_name,place_type,id,lang,...,source,tweet,hashtags,sensitive,urls,context_text,context_probability,context_type,Preprocessed_Data,Date_Only
12546,123851279.0,2022-02-19 12:59:42+00:00,,,,,,,1495020266862174208.0,en,...,Twitter for iPhone,They are so desperate they have let loose #Yat...,"YatiNarsinghanand,PlotToKillModi,hijab",False,,,0.0,,"[desperate, let, loose, yatinarsinghanand, vio...",2022-02-19
8192,3187630801,2022-02-20 00:55:03+00:00,,,,,,,1495200290210910209,en,...,Twitter for Android,No Woman is Forced to wear and Not to wear Som...,"hijab,HijabIsOurPride,HijabIsFundamentalRight,...",False,,,0.0,,"[woman, forced, wear, wear, something, hijab, ...",2022-02-20
8198,2443976946,2022-02-20 00:14:25+00:00,,,,,,,1495190066112651265,en,...,Twitter Web App,Protests in US cities against #Karnataka #hija...,"Karnataka,hijab,Islamophobic,BJP",False,"https://t.co/957kBM8AJT,https://t.co/5gY741DZP...",,0.0,,"[protest, u, city, karnataka, hijab, ban, indi...",2022-02-20
8278,1483569403425959939,2022-02-20 01:06:11+00:00,18810aa5b43e76c7,Verenigde Staten,US,"Dallas, TX",Dallas,city,1495203094224904192,en,...,Twitter for iPhone,Protest by local Dallas Muslims at JFK square ...,hijab,False,https://t.co/KCm413Yb8N,,0.0,,"[protest, local, dallas, muslim, jfk, square, ...",2022-02-20
8280,18931596,2022-02-20 01:55:42+00:00,,,,,,,1495215554508075011,en,...,Twitter Web App,Feel sorry for #zahirawasim. Religion must ma...,"zahirawasim,hijab",False,,,0.0,,"[feel, sorry, zahirawasim, religion, must, mak...",2022-02-20


In [9]:
for tweet in df_query['Preprocessed_Data']:
  tweet_query.extend(tweet)

In [10]:
tweet_query

['desperate',
 'let',
 'loose',
 'yatinarsinghanand',
 'violent',
 'rhetoric',
 'plottokillmodi',
 'hijab',
 'hate',
 'hateâ\x80¦',
 'yatinarsinghanand',
 'plottokillmodi',
 'hijab',
 'woman',
 'forced',
 'wear',
 'wear',
 'something',
 'hijab',
 'hijabisourpride',
 'hijabisfundamentalright',
 'hijaboruniform',
 'hijabcontroversy',
 'hijabcircular',
 'hijabisindividualright',
 'ð\x9f§\x95',
 'hijab',
 'hijabisourpride',
 'hijabisfundamentalright',
 'hijaboruniform',
 'hijabcontroversy',
 'hijabcircular',
 'hijabisindividualright',
 'protest',
 'u',
 'city',
 'karnataka',
 'hijab',
 'ban',
 'indian',
 'american',
 'ally',
 'demonstrated',
 'islamophobic',
 'unconstitutional',
 'ban',
 'school',
 'india',
 's',
 'nationalist',
 'bjp',
 'government',
 'take',
 'action',
 'canadaâ\x9e¡ï¸\x8f',
 'http',
 'tco957kbm8ajt',
 'take',
 'action',
 'globallyâ\x9e¡ï¸\x8f',
 'http',
 'tco5gy741dzp1',
 'http',
 'tcobvsjhjohkc',
 'karnataka',
 'hijab',
 'islamophobic',
 'bjp',
 'protest',
 'local',
 '

In [11]:
tweet_keywords = []
kw_extractor = yake.KeywordExtractor(top=20, stopwords=None)
keywords = kw_extractor.extract_keywords(' '.join(tweet_query))
for kw, v in keywords:
  print("Keyphrase: ",kw, ": score", v)
  for key in kw.split():
    if(key not in tweet_keywords):
      tweet_keywords.append(key)

print(tweet_keywords)


Keyphrase:  hijab hijab hijab : score 3.538515818965514e-07
Keyphrase:  hijab karnataka india : score 4.4295050053984284e-07
Keyphrase:  muslim islam muslim : score 7.831784725969966e-07
Keyphrase:  hijab hijabisfundamentalright india : score 8.733034882458195e-07
Keyphrase:  hijabisourright hijabcontroversy hijab : score 1.2084471666090418e-06
Keyphrase:  karnataka india hijab : score 1.2402614015115601e-06
Keyphrase:  india hijab karnataka : score 1.2402614015115601e-06
Keyphrase:  muslim hijab started : score 1.3280528877404291e-06
Keyphrase:  hijab abaya muslim : score 1.3444833831980227e-06
Keyphrase:  hijab hijabrow http : score 1.425915298203952e-06
Keyphrase:  persecution muslim hijab : score 1.4602327688888964e-06
Keyphrase:  hijabcontroversy hijab hijabisfundamentalright : score 1.5153685192729542e-06
Keyphrase:  hijabisourright hijab http : score 1.6067046974268703e-06
Keyphrase:  wear hijab hijab : score 1.6311894877235963e-06
Keyphrase:  hijab muslim india : score 1.643581

In [12]:
docs_preprocessed = []

In [13]:
#Storing file name and data
total_documents = 0
path = '/content/drive/MyDrive/Tweelink_Dataset/Tweelink_Articles_Processed'
for filename in glob(os.path.join(path, '*')):
   with open(os.path.join(os.getcwd(), filename), 'r', encoding = 'utf-8',errors = 'ignore') as f:
     filename = os.path.basename(f.name)
     data = json.load(f)
     d_date = data["Date"]
     if(d_date=="" or d_date=="Date"):
       continue
     format = '%Y-%m-%d'
 
     d_present_date = datetime.datetime.strptime(d_date, format)
 
     if(str(d_present_date.date()) not in [str(u_present_date.date()), str(u_prev_date.date()), str(u_next_date.date())]):
       continue
   
     docs_preprocessed.append({'Name':filename, 'Data':data})
     total_documents+=1
print(total_documents)

295


In [14]:
def get_relevant_docs_list_for_base_hashtag(base_hashtag, base_date, docs_preprocessed):
  relevant_docs_list = []
  for doc in docs_preprocessed:
    if doc['Data']['Base Hashtag']==base_hashtag:
      current_date = datetime.datetime.strptime(base_date, format)
      prev_date = current_date - datetime.timedelta(days=1)
      next_date = current_date + datetime.timedelta(days=1)
      if(doc['Data']['Date'] in [str(prev_date.date()), str(current_date.date()), str(next_date.date())]):
        relevant_docs_list.append(doc['Name'])
  return relevant_docs_list

In [15]:
def precision_at_k(k, base_hashtag, base_date, prediction_list, docs_preprocessed):
  relevant_docs_list = get_relevant_docs_list_for_base_hashtag(base_hashtag, base_date, docs_preprocessed)
  num_of_relevant_results=0
  for itr in range(k):
    if (prediction_list[itr][0] in relevant_docs_list):
      num_of_relevant_results+=1
  return num_of_relevant_results/k

In [16]:
def mean_average_precision(max_k, base_hashtag, base_date, relevant_docs, docs_preprocessed):
  average_precision=0
  ctr=0
  for k_val in range(1,max_k+1):
    ctr+=1
    precision_at_k_val = precision_at_k(k_val, base_hashtag, base_date, relevant_docs, docs_preprocessed)
    #print('Hashtag: {}   Precision@{}: {}'.format(base_hashtag, k_val, precision_at_k_val))
    average_precision += precision_at_k_val
  return average_precision/ctr

In [17]:
def recall_at_k(k, base_hashtag, base_date, prediction_list, docs_preprocessed):
  relevant_docs_list = get_relevant_docs_list_for_base_hashtag(base_hashtag, base_date, docs_preprocessed)
  current_num_of_relevant_results=0
  for itr in range(k):
    if (prediction_list[itr][0] in relevant_docs_list):
      current_num_of_relevant_results+=1
  if(len(relevant_docs_list)==0):
    return 0
  return current_num_of_relevant_results/len(relevant_docs_list)

In [18]:
def mean_average_recall(max_k, base_hashtag, base_date, relevant_docs, docs_preprocessed):
  average_recall=0
  ctr=0
  for k_val in range(1,max_k+1):
    ctr+=1
    recall_at_k_val = recall_at_k(k_val, base_hashtag, base_date, relevant_docs, docs_preprocessed)
    #print('Hashtag: {}   Recall@{}: {}'.format(base_hashtag, k_val, recall_at_k_val))
    average_recall += recall_at_k_val
  return average_recall/ctr

In [19]:
import operator

def find_relevant_documents(docs_preprocessed, processed_query):
  jaccard_coefficients = {}
  for document in docs_preprocessed:
    doc_text = set(document['Data']['Body_processed'])
    query = set(processed_query)
    jaccard_coefficients[document['Name']] = len(doc_text.intersection(query))/len(doc_text.union(query))
  relevant_docs = list( sorted(jaccard_coefficients.items(), key=operator.itemgetter(1),reverse=True))[:20]
  for i in range(len(relevant_docs)):
    for j in range(len(docs_preprocessed)):
      if(relevant_docs[i][0] == docs_preprocessed[j]['Name']):
        relevant_docs[i] = (relevant_docs[i][0], relevant_docs[i][1], docs_preprocessed[j]['Data']['Date'] )

  return relevant_docs



In [20]:
# with yake
# relevant_docs = find_relevant_documents(docs_preprocessed, tweet_keywords)

# without yake/ keyword extraction
relevant_docs = find_relevant_documents(docs_preprocessed, tweet_query)

In [21]:
relevant_docs

[('WriddhimanSaha_133.json', 0.08778625954198473, '2022-02-20'),
 ('PunjabElections2022_127.json', 0.08217317487266554, '2022-02-20'),
 ('PunjabElections2022_124.json', 0.06238587948874011, '2022-02-20'),
 ('UkraineRussiaCrisis_220.json', 0.059113300492610835, '2022-02-18'),
 ('vaccine_299.json', 0.058758314855875834, '2022-02-19'),
 ('UkraineRussiaCrisis_212.json', 0.05858511422254974, '2022-02-18'),
 ('MultiverseOfMadness_16.json', 0.058354476259514315, '2022-02-18'),
 ('BJPwinningUP_63.json', 0.057449253159708925, '2022-02-18'),
 ('OperationDudula_261.json', 0.05601503759398496, '2022-02-20'),
 ('ScottyFromWelding_170.json', 0.054324517512508934, '2022-02-20'),
 ('UkraineRussiaCrisis_261.json', 0.05400572246065808, '2022-02-19'),
 ('Cyberpunk2077_58.json', 0.053597650513950074, '2022-02-18'),
 ('hijab_284.json', 0.053578262533486416, '2022-02-19'),
 ('CallTheMidwife_251.json', 0.053103964098728494, '2022-02-20'),
 ('UkraineRussiaCrisis_264.json', 0.05204043429427181, '2022-02-19'),


In [22]:
mean_average_precision_hashtag = mean_average_precision(20, u_base_hashtag, u_time, relevant_docs, docs_preprocessed)
print(mean_average_precision_hashtag)

0.038701982142384614


In [23]:
mean_average_recall_hashtag = mean_average_recall(20, u_base_hashtag, u_time, relevant_docs, docs_preprocessed)
print(mean_average_recall_hashtag)

0.032499999999999994


Across all hashtags

In [29]:
global_list = [['narendramodi', '2022-02-14', 'india'],['UkraineRussiaCrisis', '2022-02-14', 'ukraine'],['IPL', '2022-02-14', 'india'],['TaylorSwift', '2022-02-14', 'USA'],['IndiaFightsCorona', '2022-02-14', 'india'],['narendramodi', '2022-02-15', 'india'],['UkraineRussiaCrisis', '2022-02-15', 'ukraine'],['IPL', '2022-02-15', 'india'],['TaylorSwift', '2022-02-15', 'USA'],['IndiaFightsCorona', '2022-02-15', 'india'],['narendramodi', '2022-02-16', 'india'],['UkraineRussiaCrisis', '2022-02-16', 'ukraine'],['IPL', '2022-02-16', 'india'],['TaylorSwift', '2022-02-16', 'USA'],['IndiaFightsCorona', '2022-02-16', 'india'],['narendramodi', '2022-02-17', 'india'],['UkraineRussiaCrisis', '2022-02-17', 'ukraine'],['IPL', '2022-02-17', 'india'],['TaylorSwift', '2022-02-17', 'USA'],['IndiaFightsCorona', '2022-02-17', 'india'],['narendramodi', '2022-02-18', 'india'],['UkraineRussiaCrisis', '2022-02-18', 'ukraine'],['IPL', '2022-02-18', 'india'],['TaylorSwift', '2022-02-18', 'USA'],['IndiaFightsCorona', '2022-02-18', 'india'],['narendramodi', '2022-02-19', 'india'],['UkraineRussiaCrisis', '2022-02-19', 'ukraine'],['IPL', '2022-02-19', 'india'],['hijab', '2022-02-19', 'india'],['vaccine', '2022-02-19', 'india'],['MillionAtIndiaPavilion', '2022-02-14', 'UAE'],['PunjabPanjeNaal', '2022-02-14', 'India'],['Euphoria', '2022-02-14', 'World'],['OscarsFanFavorite', '2022-02-14', 'World'],['ShameOnBirenSingh', '2022-02-14', 'india'],['BappiLahiri', '2022-02-16', 'india'],['BlandDoritos', '2022-02-16', 'USA'],['VERZUZ', '2022-02-16', 'USA'],['DragRaceUK', '2022-02-16', 'United Kingdom'],['BoycottWalgreens', '2022-02-18', 'USA'],['PunjabElections2022', '2022-02-20', 'india'],['WriddhimanSaha', '2022-02-20', 'india'],['stormfranklin', '2022-02-20', 'USA'],['QueenElizabeth', '2022-02-20', 'United Kingdom'],['ScottyFromWelding', '2022-02-20', 'Australia'],['CarabaoCupFinal', '2022-02-27', 'London'],['NZvSA', '2022-02-28', 'New Zealand'],['IPCC', '2022-02-28', 'Worldwide'],['SuperBowl', '2022-02-14', 'USA'],['MultiverseOfMadness', '2022-02-14', 'USA'],['Eminem', '2022-02-14', 'USA'],['IPLAuction', '2022-02-14', 'india'],['JohnsonOut21', '2022-02-14', 'United Kingdom'],['Cyberpunk2077', '2022-02-15', 'Worldwide'],['Wordle242', '2022-02-15', 'Worldwide'],['DeepSidhu', '2022-02-15', 'india'],['CanadaHasFallen', '2022-02-15', 'canada'],['IStandWithTrudeau', '2022-02-15', 'canada'],['CNNPHVPDebate', '2022-02-26', 'philippines'],['qldfloods', '2022-02-26', 'australia'],['Eurovision', '2022-02-26', 'worldwide'],['IndiansInUkraine', '2022-02-26', 'india'],['PritiPatel', '2022-02-26', 'united kingdom'],['TaylorCatterall', '2022-02-27', 'united kingdom'],['PSLFinal', '2022-02-27', 'pakistan'],['AustraliaDecides', '2022-02-27', 'australia'],['WorldNGODay', '2022-02-27', 'worldwide'],['TheBatman', '2022-02-28', 'USA'],['NationalScienceDay', '2022-02-28', 'india'],['msdtrong', '2022-02-14', 'india'],['Boycott_ChennaiSuperKings', '2022-02-14', 'india'],['GlanceJio', '2022-02-14', 'india'],['ArabicKuthu', '2022-02-14', 'india'],['Djokovic', '2022-02-15', 'australia'],['Real Madrid', '2022-02-15', 'santiago'],['bighit', '2022-02-15', 'korea'],['Maxwell', '2022-02-15', 'australia'],['mafsau', '2022-02-16', 'australia'],['channi', '2022-02-16', 'punjab'],['ayalaan', '2022-02-16', 'india'],['jkbose', '2022-02-16', 'india'],['HappyBirthdayPrinceSK', '2022-02-16', 'india'],['RandomActsOfKindnessDay', '2022-02-17', 'worldwide'],['happybirthdayjhope', '2022-02-17', 'korea'],['mohsinbaig', '2022-02-17', 'pakistan'],['aewdynamite', '2022-02-17', 'worldwide'],['aaraattu', '2022-02-17', 'india'],['ShivajiJayanti', '2022-02-18', 'india'],['PlotToKillModi', '2022-02-18', 'india'],['NationalDrinkWineDay', '2022-02-18', 'usa'],['HorizonForbiddenWest', '2022-02-18', 'worldwide'],['BoycottWalgreens', '2022-02-18', 'usa'],['CallTheMidwife', '2022-02-20', 'worldwide'],['OperationDudula', '2022-02-20', 'south africa'],['truthsocial', '2022-02-21', 'usa'],['nbaallstar', '2022-02-21', 'usa'],['shivamogga', '2022-02-21', 'india'],['HalftimeShow', '2022-02-14', 'usa'],['OttawaStrong', '2022-02-14', 'canada'],['DrDre', '2022-02-14', 'usa'],['BattleOfBillingsBridge', '2022-02-14', 'usa'],['FullyFaltooNFTdrop', '2022-02-14', 'worldwide'],['AK61', '2022-02-15', 'india'],['sandhyamukherjee', '2022-02-15', 'india'],['MUNBHA', '2022-02-15', 'worldwide'],['nursesstrike', '2022-02-15', 'australia'],['Realme9ProPlus', '2022-02-16', 'worldwide'],['KarnatakaHijabControversy', '2022-02-16', 'india'],['BJPwinningUP', '2022-02-16', 'india'],['Punjab_With_Modi', '2022-02-16', 'india'],['PushpaTheRule', '2022-02-16', 'india'],['RehmanMalik', '2022-02-22', 'india'],['harisrauf', '2022-02-22', 'pakistan'],['Rosettenville', '2022-02-22', 'south africa'],['NFU22', '2022-02-22', 'worldwide'],['justiceforharsha', '2022-02-22', 'india'],['wordle251', '2022-02-24', 'worldwide'],['ARSWOL', '2022-02-24', 'worldwide'],['stopwar', '2022-02-24', 'worldwide'],['PrayForPeace', '2022-02-24', 'worldwide'],['StopPutinNOW', '2022-02-24', 'worldwide'],['TeamGirlsCup', '2022-02-25', 'worldwide'],['Canucks', '2022-02-25', 'worldwide'],['PinkShirtDay', '2022-02-25', 'canada'],['superrugbypacific', '2022-02-25', 'australia']]

In [ ]:
global_average_mean_average_precision = []
global_mean_average_recall = []

for iter in tqdm(range(len(global_list))):
  u_base_hashtag = global_list[iter][0]
  u_time = global_list[iter][1]
  u_location = global_list[iter][2]
  tweet_query = []
  format = '%Y-%m-%d'
  u_present_date = datetime.datetime.strptime(u_time, format)
  u_prev_date = u_present_date - datetime.timedelta(days=1)
  u_next_date = u_present_date + datetime.timedelta(days=1)
  df_query = df.loc[df['hashtags'].str.contains(u_base_hashtag) & df['Date_Only'].isin([str(u_present_date.date()), str(u_prev_date.date()), str(u_next_date.date())])]

  for tweet in df_query['Preprocessed_Data']:
    tweet_query.extend(tweet)
  
  tweet_keywords = []
  kw_extractor = yake.KeywordExtractor(top=20, stopwords=None)
  keywords = kw_extractor.extract_keywords(' '.join(tweet_query))
  for kw, v in keywords:
    #print("Keyphrase: ",kw, ": score", v)
    for key in kw.split():
      if(key not in tweet_keywords):
        tweet_keywords.append(key)
  
  docs_preprocessed = []

  total_documents = 0
  path = '/content/drive/MyDrive/Tweelink_Dataset/Tweelink_Articles_Processed'
  for filename in glob(os.path.join(path, '*')):
    with open(os.path.join(os.getcwd(), filename), 'r', encoding = 'utf-8',errors = 'ignore') as f:
      filename = os.path.basename(f.name)
      data = json.load(f)
      d_date = data["Date"]
      if(d_date=="" or d_date=="Date"):
        continue
      format = '%Y-%m-%d'
  
      d_present_date = datetime.datetime.strptime(d_date, format)
  
      if(str(d_present_date.date()) not in [str(u_present_date.date()), str(u_prev_date.date()), str(u_next_date.date())]):
        continue
    
      docs_preprocessed.append({'Name':filename, 'Data':data})
      total_documents+=1
  
  # with yake
  # relevant_docs = find_relevant_documents(docs_preprocessed, tweet_keywords)

  # without yake/ keyword extraction
  relevant_docs = find_relevant_documents(docs_preprocessed, tweet_query)

  mean_average_precision_hashtag = mean_average_precision(20, u_base_hashtag, u_time, relevant_docs, docs_preprocessed)
  global_average_mean_average_precision.append(mean_average_precision_hashtag)
  #print(mean_average_precision_hashtag)

  mean_average_recall_hashtag = mean_average_recall(20, u_base_hashtag, u_time, relevant_docs, docs_preprocessed)
  global_mean_average_recall.append(mean_average_recall_hashtag)
  #print(mean_average_recall_hashtag)

In [31]:
overall_average_mean_average_precision = sum(global_average_mean_average_precision)/len(global_average_mean_average_precision)
print(overall_average_mean_average_precision)

0.2615994433668415


In [32]:
overall_mean_average_recall = sum(global_mean_average_recall)/len(global_mean_average_recall)
print(overall_mean_average_recall)

0.1709248828834454
